In [1]:
#import libraries
import sys
import plotly
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import numpy as np
import networkx as nx
import math
import time
import datetime as dt
from pivottablejs import pivot_ui
from IPython.display import HTML
from IPython.display import Image



#plotly api info
plotly.tools.set_credentials_file(username='', api_key='')

%matplotlib inline

# DataFrame

In [2]:
hospital_data = pd.read_csv('newhospitaldata.csv')
hospital_data['date'] = pd.to_datetime(hospital_data['date'],unit='s')
hospital_data['end_date'] = pd.to_datetime(hospital_data['end_date'],unit='s')
hospital_data['staff_id'] = hospital_data['staff_id'].fillna(0).astype(int)
hospital_data['staff_id'] = hospital_data['staff_id'].astype(int)
hospital_data = hospital_data.dropna(subset=['staff', 'staff_id'])


hospital_data.sort_values(by='date', ascending=True)
hospital_data.insert(2, column='date_diff', value=0)
hospital_data['date_diff'] = hospital_data['end_date'] - hospital_data['date']
hospital_data = hospital_data.sort_values(by='date', ascending=True)
hospital_data.reset_index(drop=True)


,date,end_date,date_diff,staff,staff_id,staff_position,shift,service,location,activity,status
0,2015-03-02 19:30:00,2015-03-02 23:30:00,04:00:00,Dr Ashik Hayat,41992,S1,Morning session,Weekly Roster,Medical Wards,Ward Round,Active
1,2015-03-02 19:30:00,2015-03-03 03:00:00,07:30:00,Brigitte Hollander,49964,R5,NaN,NaN,Taranaki DHB,On Call,Active
2,2015-03-02 19:30:00,2015-03-03 03:00:00,07:30:00,Jeffrey Sebastian,41994,R9,NaN,NaN,Taranaki DHB,On Call,Active
3,2015-03-02 19:30:00,2015-03-03 03:00:00,07:30:00,Sean Lance,39347,R8,NaN,NaN,Taranaki DHB,On Call,Active
4,2015-03-02 19:30:00,2015-03-03 03:00:00,07:30:00,Rodney Mbizvo,93917,R2,NaN,NaN,Taranaki DHB,On Call,Active
5,2015-03-02 19:30:00,2015-03-03 03:00:00,07:30:00,Anna Fischer,93920,R9,NaN,NaN,Taranaki DHB,On Call,Active
6,2015-03-02 19:30:00,2015-03-02 23:30:00,04:00:00,Jeffrey Sebastian,41994,R9,Morning session,Weekly Roster,Medical Wards,Ward Round,Active
7,2015-03-03 00:30:00,2015-03-03 04:00:00,03:30:00,Dr Laird Madison,44818,S5,Afternoon session,Weekly Roster,Medical OPD,OPD clinic,Active
8,2015-03-03 00:30:00,2015-03-03 04:00:00,03:30:00,Dr Campbell White,44671,S3,Afternoon session,Weekly Roster,Doctor office,CPD,Active
9,2015-03-03 00:30:00,2015-03-03 04:00:00,03:30:00,Dr Ashik Hayat,41992,S1,Afternoon session,Weekly Roster,Surgical OPD,OPD clinic,Active


# What were the most/least procedures performed?

In [3]:
#Isolate activity column - perform count 
procedure = hospital_data['activity'].value_counts()
#Convert series to dataframe
procedure = procedure.to_frame()
#Fix index
procedure = procedure.reset_index()
#Rename columns in frame to correct names
procedure.columns = ['Activity','Count']


#Plot bar graph for most/least procedures
layout = go.Layout(
    barmode='',
    title='Most/Least Procedures',
    xaxis=dict(title='Procedure'),
    yaxis=dict(title='Count')
)
data = [ go.Bar(
    x=procedure['Activity'],
    y=procedure['Count']
)]

fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

# What locations were used the most and the least?

In [4]:
#Isolate location column - perform count 
location = hospital_data['location'].value_counts()
#Convert series to dataframe
location = location.to_frame()
#Fix index
location = location.reset_index()
#Rename columns in frame to correct names
location.columns = ['Location','Count']


#Plot line graph for most/least location
N = 500
x = np.linspace(0, 1, N)
y = np.random.randn(N)
df = pd.DataFrame({'x': x, 'y': y})
df.head()

data = [
    go.Scatter(
        x=location['Location'],
        y=location['Count']
    )
]

layout = dict(title = 'Most/Least locations used',
              xaxis = dict(title = 'Locations'),
              yaxis = dict(title = 'Count')
              )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-line')

# Was staff fairly and equitably assigned?

In [5]:
#staff_assign_graph = hospital_data['staff'].value_counts()
#staff_assign_graph

# Pivot table

In [6]:
#Create pivot table
pivot_ui(hospital_data)

In [8]:
HTML('<img src="animation.gif">')

![alt text](Staff_Activity_Count2.png "Activity Count")

![alt text](Staff_Shift_Duration2.png "Shift Duration")